In [1]:
import torch
from torch.utils.data import DataLoader
from state_quantization.dataset import load_dataset
from state_quantization.dataset import DynamicsModelDataset
from state_quantization.forcasting_models import LSTMForcasting
from state_quantization.quantization_models import DiscAutoEncoder
from state_quantization.forcasting_quantization_models import ForcastingQuant
from state_quantization.trainer import ForcastingQuantTrainer
from torch.optim.lr_scheduler import MultiStepLR
from state_quantization.train import train_model, test_step
from state_quantization.eval import eval_model, compare_models

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(f"Using Device: {device}")
torch.backends.cudnn.benchmark = True

Using Device: cuda:0


## Load Dataset

In [3]:
dataset_input_key = 'merged_input'
dataset_output_key = 'merged_output'
dataset_file_path = 'tmp/ib-out/ib-samples-la.npy'
normalized_data_params_save_path = 'state_quantization/NormalizeInputConfigs.pkl'
dataset_device = 'cpu'
y_indexes = [4, 6]

train_dataset, val_dataset = load_dataset(file_path=dataset_file_path, input_key=dataset_input_key,
                                          output_key=dataset_output_key, dataset_class=DynamicsModelDataset,
                                          normalize=True, device=dataset_device, y_clip_range=y_indexes,
                                          normalized_data_params_save_path=normalized_data_params_save_path)

batch_size = 8000
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 0,
          'drop_last': True,
          'pin_memory': not train_dataset.x.is_cuda}
print(params)
train_loader = DataLoader(train_dataset, **params)
val_loader = DataLoader(val_dataset, **params)
train_dataset.y[0]

Clipping y
torch.Size([672000, 39, 6])
torch.Size([672000, 10, 2])
torch.Size([288000, 39, 6])
torch.Size([288000, 10, 2])
{'batch_size': 8000, 'shuffle': True, 'num_workers': 0, 'drop_last': True, 'pin_memory': True}


tensor([[-0.9307,  0.4514],
        [-0.9254,  0.4017],
        [-1.0641,  0.3700],
        [-1.0641,  0.3580],
        [-0.9195,  0.4085],
        [-0.9129,  0.4301],
        [-0.8791,  0.3999],
        [-1.0641,  0.3301],
        [-1.0641,  0.3159],
        [-0.9353,  0.3056]])

## Define model

In [4]:
model_path = 'state_quantization/model'

num_of_features = train_dataset.get_features_size()
seq_len = train_dataset.get_seq_len()
hidden_size = 16
out_size = train_dataset.get_output_feature_size()
print(f'Out Size:{out_size}')
look_ahead = train_dataset.get_look_ahead_size()
n_layers = 1
dropout = 0.2

forcasting_model = LSTMForcasting(features=num_of_features, hidden_size=hidden_size, out_size=out_size, seq_len=seq_len,
                                  look_ahead=look_ahead, dropout=dropout, n_layers=n_layers)

disc_autoencoder_input_size = hidden_size * 2
bottleneck_size = 10
ae_dropout = 0.2
disc_autoencoder = DiscAutoEncoder(input_size=disc_autoencoder_input_size, bottleneck_size=bottleneck_size,
                                   dropout=ae_dropout)

model = ForcastingQuant(forcasting_model=forcasting_model, autoencoder_quant_model=disc_autoencoder).to(device=device)

load_to_gpu = model.is_cuda() and not train_dataset.x.is_cuda
print(load_to_gpu)



Out Size:2
LSTM Layers
ModuleList(
  (0): LSTMCell(6, 16)
)
LSTM Dropout Layers
ModuleList()
Fully Connected Layers
ModuleList(
  (0): Linear(in_features=16, out_features=16, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=16, out_features=16, bias=True)
  (4): GELU(approximate=none)
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=16, out_features=2, bias=True)
)
[32, 16]
[16, 32]
Encoder Layers
ModuleList(
  (0): Linear(in_features=32, out_features=32, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): GELU(approximate=none)
  (5): Dropout(p=0.2, inplace=False)
)
Bottleneck Layers
ModuleList(
  (0): Linear(in_features=16, out_features=10, bias=True)
  (1): StraightThroughEstimator()
)
Decoded Layers
ModuleList(
  (0): Linear(in_features=10, out_features=16, bias=True)
  (1): GELU(approximate=none)
  (2): Dropout(p=0.2, inpla

## Define Trainer

In [5]:
gamma = 0.1

forecasting_learning_rate = 1e-3
autoencoder_learning_rate = 1e-3

forecasting_lr_milestones = [90]
autoencoder_lr_milestones = [35]
forecasting_optimizer = torch.optim.Adam(model.forcasting_model.parameters(),
                                         lr=forecasting_learning_rate)
autoencoder_optimizer = torch.optim.Adam(model.autoencoder_quant_model.parameters(),
                                         lr=autoencoder_learning_rate)
forecasting_lr_scheduler = MultiStepLR(forecasting_optimizer, milestones=forecasting_lr_milestones, gamma=gamma)
autoencoder_lr_scheduler = MultiStepLR(autoencoder_optimizer, milestones=autoencoder_lr_milestones, gamma=gamma)

trainer = ForcastingQuantTrainer(forcasting_quant_model=model, train_loader=train_loader, test_loader=val_loader,
                                 load_to_gpu=load_to_gpu, forecasting_optimizer=forecasting_optimizer,
                                 forecasting_lr_scheduler=forecasting_lr_scheduler,
                                 autoencoder_lr_scheduler=autoencoder_lr_scheduler,
                                 autoencoder_optimizer=autoencoder_optimizer)



## Train Model

In [ ]:
trainer.train(n_epochs=200)

Untrained test
--------
--------------------------------------
Forcasting Test loss: 1.01047596666548
Autoencoder Test loss: 0.09716833237972525
--------------------------------------
--------------------------------------
Epoch 1
---------
--------------------------------------
Forcasting Train loss: 0.8297108056999388
Autoencoder Train loss: 0.2127241369869028
--------------------------------------
Forcasting Test loss: 0.4643946381078826
Autoencoder Test loss: 0.3201235623823272
--------------------------------------
Forecasting lr: [0.001]
Autoencoder lr: [0.001]
Epoch time: epoch_time = 8.284s
--------------------------------------
--------------------------------------
Epoch 2
---------
--------------------------------------
Forcasting Train loss: 0.3210209256836346
Autoencoder Train loss: 0.21653443432989575
--------------------------------------
Forcasting Test loss: 0.1277384561383062
Autoencoder Test loss: 0.13479670468303892
--------------------------------------
Forecasting

## Save Model

In [ ]:
torch.save(model, model_path)
torch.cuda.empty_cache()
del model